In [46]:
!pip install pymysql sqlalchemy

# 필요 라이브러리 import

In [47]:
import requests
import bs4
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from sqlalchemy import create_engine, inspect

## 수정 포인트 (공공데이터 포털 API Key & DB 연결 정보)

In [ ]:
# API Key 설정
API_Key = unquote("{API Key}")

# DB 연결 정보
user = '{user}'
password = '{password}'
host = '{host}'
port = '{port}'
database = "{database}"

# 연도별 자연재해 피해액 데이터 수집 함수

In [49]:
def collect_disaster_year(inYear):
    """
    함수정의: 행정안전부_통계연보_연도별 자연재해 피해 API를 통해 파라미터 연도의 데이터를 불러와 DB로 저장하는 함수
    입력 파라미터 : inYear -> str
    출력 결과: 없음 (결과는 'disaster_damage_by_year' 테이블에 저장됨)
    """

    # 수집중 문구 출력
    print(f"{inYear}년까지의 연도별 자연재해 피해액 데이터 수집중...")

    # 행정안전부_통계연보_연도별 자연재해 피해 API URL
    disaster_year_url = "http://apis.data.go.kr/1741000/NaturalDisasterDamageByYear/getNaturalDisasterDamageByYear"

    # 쿼리 파라미터에 API_Key 추가
    queryParams = '?' + urlencode({
                        quote_plus('ServiceKey') : API_Key,
                    })

    # requests 라이브러리를 사용하여 API로부터 XML 데이터를 가져옴
    response = requests.get(disaster_year_url+queryParams)

    # 응답 인코딩을 UTF-8로 설정
    response.encoding = "utf-8"

    # lxml-xml 파서를 사용하여 BeautifulSoup으로 XML 파싱
    disaster_year_xmlobj = bs4.BeautifulSoup(response.text, "lxml-xml")

    # 각 재해 항목별 데이터를 XML에서 추출해 리스트로 저장
    year_list = disaster_year_xmlobj.find_all("wrttimeid") # 연도
    total_list = disaster_year_xmlobj.find_all("tot") # 합계
    typhoon_list = disaster_year_xmlobj.find_all("typhoon") # 태풍
    heavy_rain_list = disaster_year_xmlobj.find_all("heavy_rain") # 호우
    heavy_snow_list = disaster_year_xmlobj.find_all("heavy_snow") # 대설
    heavy_wind_list = disaster_year_xmlobj.find_all("heavy_wind") # 강풍
    wind_wave_strong_wind_list = disaster_year_xmlobj.find_all("wind_wave_strong_wind") # 풍랑·강풍
    typhoon_heavy_rain_list = disaster_year_xmlobj.find_all("typhoon_heavy_rain") # 태풍·호우
    lightning_list = disaster_year_xmlobj.find_all("lightning") # 낙뢰
    cold_wave_list = disaster_year_xmlobj.find_all("cold_wave") # 한파
    earthquak_list = disaster_year_xmlobj.find_all("earthquak") # 지진
    heatwave_list = disaster_year_xmlobj.find_all("heatwave") # 폭염

    # 행 별로 결과를 저장할 행 리스트 선언
    rowList = []

    # 데이터프레임의 컬럼명 정의
    columnNameList = ["year", "total", "typhoon", "heavy_rain", "heavy_snow", "heavy_wind", "wind_wave_strong_wind", "typhoon_heavy_rain", "lightning", "cold_wave", "earthquake", "heatwave"]

    # 각 항목별 값을 추출하여 리스트로 저장
    for i in range(len(year_list)):
        year = year_list[i].text
        total = total_list[i].text
        typhoon = typhoon_list[i].text
        heavy_rain = heavy_rain_list[i].text
        heavy_snow = heavy_snow_list[i].text
        heavy_wind = heavy_wind_list[i].text
        wind_wave_strong_wind = wind_wave_strong_wind_list[i].text
        typhoon_heavy_rain = typhoon_heavy_rain_list[i].text
        lightning = lightning_list[i].text
        cold_wave = cold_wave_list[i].text
        earthquak = earthquak_list[i].text
        heatwave = heatwave_list[i].text

        # 각 항목을 한 행으로 리스트에 추가
        rowList.append([year, total, typhoon, heavy_rain, heavy_snow, heavy_wind, wind_wave_strong_wind, typhoon_heavy_rain, lightning, cold_wave, earthquak, heatwave])

    # pandas DataFrame으로 변환
    disaster_year_result = pd.DataFrame(rowList, columns = columnNameList)

    # 모든 컬럼을 숫자형(int)으로 변환
    disaster_year_result = disaster_year_result.astype(int)

    # 수집 가능한 최소 연도 저장
    min_year = disaster_year_result["year"].min()

    # inYear보다 작거나 같은 데이터만 필터링
    disaster_year_result = disaster_year_result[disaster_year_result["year"] <= inYear]

    # 필터링 결과가 없으면 메시지 출력 후 함수 종료
    if disaster_year_result.empty:
        print(f"해당 연도의 데이터가 존재하지 않습니다. (수집 가능한 최소 연도: {min_year}년)")
        return

    # MySQL 데이터베이스 연결 설정
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_disabled=True')

    # 저장할 테이블 이름
    table_name = "disaster_damage_by_year"

    # DataFrame을 MySQL 테이블에 저장 (기존 테이블 존재 시 새로운 데이터 추가)
    disaster_year_result.to_sql(name = table_name, con = engine, if_exists="replace", index=False)

    # 데이터베이스 연결 종료
    engine.dispose()

    # 수집 완료 문구 출력
    print(f"{inYear}년까지의 연도별 자연재해 피해액 데이터 수집 및 DB 저장 완료!")

# 연도별 자연재해 복구비 데이터 수집 함수



In [50]:
def collect_cost_year(inYear):
    """
    함수정의: 행정안전부_통계연보_연도별 자연재해 복구비 API를 통해 파라미터 연도의 데이터를 불러와 DB로 저장하는 함수
    입력 파라미터 : inYear -> str
    출력 결과: 없음 (결과는 'disaster_damage_by_year' 테이블에 저장됨)
    """

    # 수집중 문구 출력
    print(f"{inYear}년까지의 연도별 자연재해 복구비 데이터 수집중...")

    # 행정안전부_통계연보_연도별 자연재해 복구비 API URL
    cost_year_url = "http://apis.data.go.kr/1741000/AnnualDisasterRecoveryCosts/getAnnualDisasterRecoveryCosts"

    # 쿼리 파라미터에 API_Key 추가
    queryParams = '?' + urlencode({
                        quote_plus('ServiceKey') : API_Key,
                    })

    # requests 라이브러리를 사용하여 API로부터 XML 데이터를 가져옴
    response = requests.get(cost_year_url+queryParams)

    # 응답 인코딩을 UTF-8로 설정
    response.encoding = "utf-8"

    # lxml-xml 파서를 사용하여 BeautifulSoup으로 XML 파싱
    cost_year_xmlobj = bs4.BeautifulSoup(response.text, "lxml-xml")

    # 각 재해 항목별 데이터를 XML에서 추출해 리스트로 저장
    year_list = cost_year_xmlobj.find_all("wrttimeid") # 연도
    total_list = cost_year_xmlobj.find_all("total") # 합계
    typhoon_list = cost_year_xmlobj.find_all("typhoon") # 태풍
    heavy_rain_list = cost_year_xmlobj.find_all("heavy_rain") # 호우
    heavy_snow_list = cost_year_xmlobj.find_all("heavy_snow") # 대설
    heavy_wind_list = cost_year_xmlobj.find_all("strong_wind") # 강풍
    wind_wave_strong_wind_list = cost_year_xmlobj.find_all("wind_wave_strong_wind") # 풍랑·강풍
    typhoon_heavy_rain_list = cost_year_xmlobj.find_all("typhoon_heavy_rain") # 태풍·호우
    lightning_list = cost_year_xmlobj.find_all("lightning") # 낙뢰
    cold_wave_list = cost_year_xmlobj.find_all("cold_wave") # 한파
    earthquak_list = cost_year_xmlobj.find_all("earthquake") # 지진
    heatwave_list = cost_year_xmlobj.find_all("heatwave") # 폭염

    # 행 별로 결과를 저장할 행 리스트 선언
    rowList = []

    # 데이터프레임의 컬럼명 정의
    columnNameList = ["year", "total", "typhoon", "heavy_rain", "heavy_snow", "heavy_wind", "wind_wave_strong_wind", "typhoon_heavy_rain", "lightning", "cold_wave", "earthquake", "heatwave"]

    # 각 항목별 값을 추출하여 리스트로 저장
    for i in range(len(year_list)):
        year = year_list[i].text
        total = total_list[i].text
        typhoon = typhoon_list[i].text
        heavy_rain = heavy_rain_list[i].text
        heavy_snow = heavy_snow_list[i].text
        heavy_wind = heavy_wind_list[i].text
        wind_wave_strong_wind = wind_wave_strong_wind_list[i].text
        typhoon_heavy_rain = typhoon_heavy_rain_list[i].text
        lightning = lightning_list[i].text
        cold_wave = cold_wave_list[i].text
        earthquak = earthquak_list[i].text
        heatwave = heatwave_list[i].text

        # 각 항목을 한 행으로 리스트에 추가
        rowList.append([year, total, typhoon, heavy_rain, heavy_snow, heavy_wind, wind_wave_strong_wind, typhoon_heavy_rain, lightning, cold_wave, earthquak, heatwave])

    # pandas DataFrame으로 변환
    cost_year_result = pd.DataFrame(rowList, columns = columnNameList)

    # 모든 컬럼을 숫자형(int)으로 변환
    cost_year_result = cost_year_result.astype(int)

    # 수집 가능한 최소 연도 저장
    min_year = cost_year_result["year"].min()

    # inYear보다 작거나 같은 데이터만 필터링
    cost_year_result = cost_year_result[cost_year_result["year"] <= inYear]

    # 필터링 결과가 없으면 메시지 출력 후 함수 종료
    if cost_year_result.empty:
        print(f"해당 연도의 데이터가 존재하지 않습니다. (수집 가능한 최소 연도: {min_year}년)")
        return

    # MySQL 데이터베이스 연결 설정
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_disabled=True')

    # 저장할 테이블 이름
    table_name = "recovery_cost_by_year"

    # DataFrame을 MySQL 테이블에 저장 (기존 테이블 존재 시 새로운 데이터 추가)
    cost_year_result.to_sql(name = table_name, con = engine, if_exists="replace", index=False)

    # 데이터베이스 연결 종료
    engine.dispose()

    # 수집 완료 문구 출력
    print(f"{inYear}년까지의 연도별 자연재해 복구비 데이터 수집 및 DB 저장 완료!")

# 지역별 자연재해 복구비 데이터 수집 함수

In [51]:
def collect_cost_region(inYear):
    """
    함수정의: 행정안전부_통계연보_지역별 자연재해 복구비 API를 통해 파라미터 연도의 데이터를 불러와 DB로 저장하는 함수
    입력 파라미터 : inYear -> str
    출력 결과: 없음 (결과는 'disaster_damage_by_year' 테이블에 저장됨)
    """

    # 수집중 문구 출력
    print(f"{inYear}년까지의 지역별 자연재해 복구비 데이터 수집중...")

    # 행정안전부_통계연보_지역별 자연재해 복구비 API URL
    cost_region_url = "http://apis.data.go.kr/1741000/RegionDisasterRecoveryCosts/getRegionDisasterRecoveryCosts"

    # 쿼리 파라미터에 API_Key 추가
    queryParams = '?' + urlencode({
                        quote_plus('ServiceKey') : API_Key,
                    })

    # requests 라이브러리를 사용하여 API로부터 XML 데이터를 가져옴
    response = requests.get(cost_region_url+queryParams)

    # 응답 인코딩을 UTF-8로 설정
    response.encoding = "utf-8"

    # lxml-xml 파서를 사용하여 BeautifulSoup으로 XML 파싱
    cost_region_xmlobj = bs4.BeautifulSoup(response.text, "lxml-xml")

    # 각 재해 항목별 데이터를 XML에서 추출해 리스트로 저장
    year_list = cost_region_xmlobj.find_all("wrttimeid") # 연도
    region_list = cost_region_xmlobj.find_all("region") # 지역
    total_list = cost_region_xmlobj.find_all("tot") # 합계
    typhoon_list = cost_region_xmlobj.find_all("typhoon") # 태풍
    heavy_rain_list = cost_region_xmlobj.find_all("heavy_rain") # 호우
    heavy_snow_list = cost_region_xmlobj.find_all("heavy_snow") # 대설
    heavy_wind_list = cost_region_xmlobj.find_all("heavy_wind") # 강풍
    wind_wave_strong_wind_list = cost_region_xmlobj.find_all("wind_wave_strong_wind") # 풍랑·강풍
    typhoon_heavy_rain_list = cost_region_xmlobj.find_all("typhoon_heavy_rain") # 태풍·호우
    lightning_list = cost_region_xmlobj.find_all("lightning") # 낙뢰
    cold_wave_list = cost_region_xmlobj.find_all("cold_wave") # 한파
    earthquak_list = cost_region_xmlobj.find_all("earthquak") # 지진
    heatwave_list = cost_region_xmlobj.find_all("heatwave") # 폭염

    # 행 별로 결과를 저장할 행 리스트 선언
    rowList = []

    # 데이터프레임의 컬럼명 정의
    columnNameList = ["year", "region", "total", "typhoon", "heavy_rain", "heavy_snow", "heavy_wind", "wind_wave_strong_wind", "typhoon_heavy_rain", "lightning", "cold_wave", "earthquake", "heatwave"]

    # 각 항목별 값을 추출하여 리스트로 저장
    for i in range(len(year_list)):
        year = year_list[i].text
        region = region_list[i].text
        total = total_list[i].text
        typhoon = typhoon_list[i].text
        heavy_rain = heavy_rain_list[i].text
        heavy_snow = heavy_snow_list[i].text
        heavy_wind = heavy_wind_list[i].text
        wind_wave_strong_wind = wind_wave_strong_wind_list[i].text
        typhoon_heavy_rain = typhoon_heavy_rain_list[i].text
        lightning = lightning_list[i].text
        cold_wave = cold_wave_list[i].text
        earthquak = earthquak_list[i].text
        heatwave = heatwave_list[i].text

        # 각 항목을 한 행으로 리스트에 추가
        rowList.append([year, region, total, typhoon, heavy_rain, heavy_snow, heavy_wind, wind_wave_strong_wind, typhoon_heavy_rain, lightning, cold_wave, earthquak, heatwave])

    # pandas DataFrame으로 변환
    cost_region_result = pd.DataFrame(rowList, columns = columnNameList)

    # region 컬럼을 제외한 나머지 컬럼을 숫자형으로 변환
    cols_to_convert = cost_region_result.columns.difference(['region'])
    cost_region_result[cols_to_convert] = cost_region_result[cols_to_convert].apply(pd.to_numeric, errors='coerce')


    # 수집 가능한 최소 연도 저장
    min_year = cost_region_result["year"].min()

    # inYear보다 작거나 같은 데이터만 필터링
    cost_region_result = cost_region_result[cost_region_result["year"] <= inYear]

    # 필터링 결과가 없으면 메시지 출력 후 함수 종료
    if cost_region_result.empty:
        print(f"해당 연도의 데이터가 존재하지 않습니다. (수집 가능한 최소 연도: {min_year}년)")
        return

    # MySQL 데이터베이스 연결 설정
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_disabled=True')

    # 저장할 테이블 이름
    table_name = "recovery_cost_by_region"

    # DataFrame을 MySQL 테이블에 저장 (기존 테이블 존재 시 새로운 데이터 추가)
    cost_region_result.to_sql(name = table_name, con = engine, if_exists="replace", index=False)

    # 데이터베이스 연결 종료
    engine.dispose()

    # 수집 완료 문구 출력
    print(f"{inYear}년까지의 지역별 자연재해 복구비 데이터 수집 및 DB 저장 완료!")

# 지역별 자연재해 피해액 데이터 수집 함수

In [52]:
def collect_disaster_region(inYear):
    """
    함수정의: 행정안전부_통계연보_지역별 자연재해 피해 API를 통해 파라미터 연도의 데이터를 불러와 DB로 저장하는 함수
    입력 파라미터 : inYear -> str
    출력 결과:
    """

    # 수집중 문구 출력
    print(f"{inYear}년까지의 지역별 자연재해 피해액 데이터 수집중...")

    # 행정안전부_통계연보_지역별 자연재해 피해 API URL
    disaster_region_url = "http://apis.data.go.kr/1741000/RegionalNaturalDisasterDamage/getRegionalNaturalDisasterDamage"

    # 쿼리 파라미터에 API_Key 추가
    queryParams = '?' + urlencode({
                        quote_plus('ServiceKey') : API_Key,
                    })

    # requests 라이브러리를 사용하여 API로부터 XML 데이터를 가져옴
    response = requests.get(disaster_region_url+queryParams)

    # 응답 인코딩을 UTF-8로 설정
    response.encoding = "utf-8"

    # lxml-xml 파서를 사용하여 BeautifulSoup으로 XML 파싱
    disaster_region_xmlobj = bs4.BeautifulSoup(response.text, "lxml-xml")

    # 각 재해 항목별 데이터를 XML에서 추출해 리스트로 저장
    year_list = disaster_region_xmlobj.find_all("wrttimeid") # 연도
    region_list = disaster_region_xmlobj.find_all("region") # 지역

    prop_tot_list = disaster_region_xmlobj.find_all("prop_tot") # 합계_재산
    life_tot_list = disaster_region_xmlobj.find_all("life_tot") # 합계_인명

    typhoon_prop_list = disaster_region_xmlobj.find_all("typhoon_property_damage") # 태풍_재산
    typhoon_life_list = disaster_region_xmlobj.find_all("typhoon_life_damage") # 태풍_인명

    heavy_rain_prop_list = disaster_region_xmlobj.find_all("heavy_rain_property_damage") # 호우_재산
    heavy_rain_life_list = disaster_region_xmlobj.find_all("heavy_rain_life_damage") # 호우_인명

    heavy_snow_prop_list = disaster_region_xmlobj.find_all("heavy_snow_property_damage") # 대설_재산
    heavy_snow_life_list = disaster_region_xmlobj.find_all("heavy_snow_life_damage") # 대설_인명

    strong_wind_prop_list = disaster_region_xmlobj.find_all("strong_wind_property_damage") # 강풍_재산
    strong_wind_life_list = disaster_region_xmlobj.find_all("strong_wind_life_damage") # 강풍_인명

    wind_wave_strong_wind_prop_list = disaster_region_xmlobj.find_all("wind_wave_strong_wind_property_damage") # 풍랑·강풍_재산
    wind_wave_strong_wind_life_list = disaster_region_xmlobj.find_all("wind_wave_strong_wind_life_damage") # 풍랑·강풍_인명

    typhoon_heavy_rain_prop_list = disaster_region_xmlobj.find_all("typhoon_heavy_rain_property_damage") # 태풍·호우_재산
    typhoon_heavy_rain_life_list = disaster_region_xmlobj.find_all("typhoon_heavy_rain_life_damage") # 태풍·호우_인명

    lightning_prop_list = disaster_region_xmlobj.find_all("lightning_property_damage") # 낙뢰_재산
    lightning_life_list = disaster_region_xmlobj.find_all("lightning_life_damage") # 낙뢰_인명

    cold_wave_prop_list = disaster_region_xmlobj.find_all("cold_wave_property_damage") # 한파_재산
    cold_wave_life_list = disaster_region_xmlobj.find_all("cold_wave_life_damage") # 한파_인명

    earthquake_prop_list = disaster_region_xmlobj.find_all("earthquake_property_damage") # 지진_재산
    earthquake_life_list = disaster_region_xmlobj.find_all("earthquake_life_damage") # 지진_인명

    heatwave_prop_list = disaster_region_xmlobj.find_all("heatwave_property_damage") # 폭염_재산
    heatwave_life_list = disaster_region_xmlobj.find_all("heatwave_life_damage") # 폭염_인명

    # 행 별로 결과를 저장할 행 리스트 선언
    rowList = []

    # 데이터프레임의 컬럼명 정의
    columnNameList = ["year", "region", "total", "typhoon", "heavy_rain", "heavy_snow", "heavy_wind", "wind_wave_strong_wind", "typhoon_heavy_rain", "lightning", "cold_wave", "earthquake", "heatwave"]

    # 각 항목별 값을 추출하여 리스트로 저장
    for i in range(len(year_list)):
        year = year_list[i].text
        region = region_list[i].text
        total = str(float(prop_tot_list[i].text) + float(life_tot_list[i].text))

        # 각 재해별 총합 (property + life)
        typhoon = str(float(typhoon_prop_list[i].text) + float(typhoon_life_list[i].text))
        heavy_rain = str(float(heavy_rain_prop_list[i].text) + float(heavy_rain_life_list[i].text))
        heavy_snow = str(float(heavy_snow_prop_list[i].text) + float(heavy_snow_life_list[i].text))
        heavy_wind = str(float(strong_wind_prop_list[i].text) + float(strong_wind_life_list[i].text))
        wind_wave_strong_wind = str(float(wind_wave_strong_wind_prop_list[i].text) + float(wind_wave_strong_wind_life_list[i].text))
        typhoon_heavy_rain = str(float(typhoon_heavy_rain_prop_list[i].text) + float(typhoon_heavy_rain_life_list[i].text))
        lightning = str(float(lightning_prop_list[i].text) + float(lightning_life_list[i].text))
        cold_wave = str(float(cold_wave_prop_list[i].text) + float(cold_wave_life_list[i].text))
        earthquake = str(float(earthquake_prop_list[i].text) + float(earthquake_life_list[i].text))
        heatwave = str(float(heatwave_prop_list[i].text) + float(heatwave_life_list[i].text))

        # 각 항목을 한 행으로 리스트에 추가
        rowList.append([year, region, total, typhoon, heavy_rain, heavy_snow, heavy_wind, wind_wave_strong_wind, typhoon_heavy_rain, lightning, cold_wave, earthquake, heatwave])

    # pandas DataFrame으로 변환
    disaster_region_result = pd.DataFrame(rowList, columns = columnNameList)

    # region 컬럼을 제외한 나머지 컬럼을 숫자형으로 변환
    cols_to_convert = disaster_region_result.columns.difference(['region'])
    disaster_region_result[cols_to_convert] = disaster_region_result[cols_to_convert].apply(pd.to_numeric, errors='coerce')

    # 수집 가능한 최소 연도 저장
    min_year = disaster_region_result["year"].min()

    # inYear보다 작거나 같은 데이터만 필터링
    disaster_region_result = disaster_region_result[disaster_region_result["year"] <= inYear]

    # 필터링 결과가 없으면 메시지 출력 후 함수 종료
    if disaster_region_result.empty:
        print(f"해당 연도의 데이터가 존재하지 않습니다. (수집 가능한 최소 연도: {min_year}년)")
        return

    # MySQL 데이터베이스 연결 설정
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_disabled=True')

    # 저장할 테이블 이름
    table_name = "disaster_damage_by_region"

    # DataFrame을 MySQL 테이블에 저장 (기존 테이블 존재 시 새로운 데이터 추가)
    disaster_region_result.to_sql(name=table_name, con=engine, if_exists="replace", index=False)

    # 데이터베이스 연결 종료
    engine.dispose()

    # 수집 완료 문구 출력
    print(f"{inYear}년까지의 지역별 자연재해 피해액 데이터 수집 및 DB 저장 완료!")


# 연도 지정 데이터 수집 및 DB append

In [53]:
# 연도 지정
inYear = int(input("수집할 연도를 입력해주세요 (예 : 2016) : "))

# 출력 이쁘게 하기 위한 줄바꿈
print()

# 데이터 수집
collect_disaster_year(inYear)

print()

collect_cost_year(inYear)

print()

collect_cost_region(inYear)

print()

collect_disaster_region(inYear)

수집할 연도를 입력해주세요 (예 : 2016) : 2016

2016년까지의 연도별 자연재해 피해액 데이터 수집중...
2016년까지의 연도별 자연재해 피해액 데이터 수집 및 DB 저장 완료!

2016년까지의 연도별 자연재해 복구비 데이터 수집중...
2016년까지의 연도별 자연재해 복구비 데이터 수집 및 DB 저장 완료!

2016년까지의 지역별 자연재해 복구비 데이터 수집중...
2016년까지의 지역별 자연재해 복구비 데이터 수집 및 DB 저장 완료!

2016년까지의 지역별 자연재해 피해액 데이터 수집중...
2016년까지의 지역별 자연재해 피해액 데이터 수집 및 DB 저장 완료!


# 데이터 수집 과정

## 연도별 자연재해 피해액 데이터 수집

In [ ]:
# 행정안전부_통계연보_연도별 자연재해 피해 API URL
disaster_year_url = "http://apis.data.go.kr/1741000/NaturalDisasterDamageByYear/getNaturalDisasterDamageByYear"

In [ ]:
# 쿼리 파라미터에 API_Key 추가
queryParams = '?' + urlencode({
                    quote_plus('ServiceKey') : API_Key,
                })

In [ ]:
# requests 라이브러리를 사용하여 API로부터 XML 데이터를 가져옴
response = requests.get(disaster_year_url+queryParams)

# 응답 인코딩을 UTF-8로 설정
response.encoding = "utf-8"

# lxml-xml 파서를 사용하여 BeautifulSoup으로 XML 파싱
disaster_year_xmlobj = bs4.BeautifulSoup(response.text, "lxml-xml")
disaster_year_xmlobj

<?xml version="1.0" encoding="utf-8"?>
<NaturalDisasterDamageByYear>
<head>
<totalCount>7</totalCount>
<numOfRows>100</numOfRows>
<pageNo>1</pageNo>
<type>XML</type>
<RESULT>
<resultCode>INFO-0</resultCode>
<resultMsg>NOMAL SERVICE</resultMsg>
</RESULT>
</head>
<row>
<wrttimeid>2016</wrttimeid>
<seq>1</seq>
<tot>288862</tot>
<typhoon>214965</typhoon>
<heavy_rain>35887</heavy_rain>
<heavy_snow>18689</heavy_snow>
<heavy_wind>0</heavy_wind>
<wind_wave_strong_wind>8302</wind_wave_strong_wind>
<typhoon_heavy_rain>0</typhoon_heavy_rain>
<lightning>0</lightning>
<cold_wave>0</cold_wave>
<earthquak>11020</earthquak>
<heatwave>0</heatwave>
</row>
<row>
<wrttimeid>2017</wrttimeid>
<seq>1</seq>
<tot>187302</tot>
<typhoon>0</typhoon>
<heavy_rain>101592</heavy_rain>
<heavy_snow>83</heavy_snow>
<heavy_wind>0</heavy_wind>
<wind_wave_strong_wind>605</wind_wave_strong_wind>
<typhoon_heavy_rain>0</typhoon_heavy_rain>
<lightning>0</lightning>
<cold_wave>0</cold_wave>
<earthquak>85022</earthquak>
<heatwav

### 각 재해 항목별 데이터를 XML에서 추출해 리스트로 저장

In [ ]:
year_list = disaster_year_xmlobj.find_all("wrttimeid") # 연도
year_list

[<wrttimeid>2016</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2018</wrttimeid>,
 <wrttimeid>2019</wrttimeid>,
 <wrttimeid>2020</wrttimeid>,
 <wrttimeid>2021</wrttimeid>,
 <wrttimeid>2022</wrttimeid>]

In [ ]:
total_list = disaster_year_xmlobj.find_all("tot") # 합계
total_list

[<tot>288862</tot>,
 <tot>187302</tot>,
 <tot>141284</tot>,
 <tot>216226</tot>,
 <tot>1318177</tot>,
 <tot>66054</tot>,
 <tot>592656</tot>]

In [ ]:
typhoon_list = disaster_year_xmlobj.find_all("typhoon") # 태풍
typhoon_list

[<typhoon>214965</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>64200</typhoon>,
 <typhoon>212778</typhoon>,
 <typhoon>222541</typhoon>,
 <typhoon>21086</typhoon>,
 <typhoon>244046</typhoon>]

In [ ]:
heavy_rain_list = disaster_year_xmlobj.find_all("heavy_rain") # 호우
heavy_rain_list

[<heavy_rain>35887</heavy_rain>,
 <heavy_rain>101592</heavy_rain>,
 <heavy_rain>53800</heavy_rain>,
 <heavy_rain>1651</heavy_rain>,
 <heavy_rain>1095172</heavy_rain>,
 <heavy_rain>40646</heavy_rain>,
 <heavy_rain>332560</heavy_rain>]

In [ ]:
heavy_snow_list = disaster_year_xmlobj.find_all("heavy_snow") # 대설
heavy_snow_list

[<heavy_snow>18689</heavy_snow>,
 <heavy_snow>83</heavy_snow>,
 <heavy_snow>14032</heavy_snow>,
 <heavy_snow>671</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>3527</heavy_snow>,
 <heavy_snow>15439</heavy_snow>]

In [ ]:
heavy_wind_list = disaster_year_xmlobj.find_all("heavy_wind") # 강풍
heavy_wind_list

[<heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>7</heavy_wind>,
 <heavy_wind>7</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>445</heavy_wind>,
 <heavy_wind>26</heavy_wind>]

In [ ]:
wind_wave_strong_wind_list = disaster_year_xmlobj.find_all("wind_wave_strong_wind") # 풍랑·강풍
wind_wave_strong_wind_list

[<wind_wave_strong_wind>8302</wind_wave_strong_wind>,
 <wind_wave_strong_wind>605</wind_wave_strong_wind>,
 <wind_wave_strong_wind>2823</wind_wave_strong_wind>,
 <wind_wave_strong_wind>474</wind_wave_strong_wind>,
 <wind_wave_strong_wind>319</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>585</wind_wave_strong_wind>]

In [ ]:
typhoon_heavy_rain_list = disaster_year_xmlobj.find_all("typhoon_heavy_rain") # 태풍·호우
typhoon_heavy_rain_list

[<typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>6416</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>]

In [ ]:
lightning_list = disaster_year_xmlobj.find_all("lightning") # 낙뢰
lightning_list

[<lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>26</lightning>,
 <lightning>0</lightning>,
 <lightning>52</lightning>,
 <lightning>0</lightning>]

In [ ]:
cold_wave_list = disaster_year_xmlobj.find_all("cold_wave") # 한파
cold_wave_list

[<cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>145</cold_wave>,
 <cold_wave>280</cold_wave>,
 <cold_wave>0</cold_wave>]

In [ ]:
earthquak_list = disaster_year_xmlobj.find_all("earthquak") # 지진
earthquak_list

[<earthquak>11020</earthquak>,
 <earthquak>85022</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>]

In [ ]:
heatwave_list = disaster_year_xmlobj.find_all("heatwave") # 폭염
heatwave_list

[<heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>7</heatwave>,
 <heatwave>618</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>]

In [ ]:
# 행 별로 결과를 저장할 행 리스트 선언
rowList = []

# 데이터프레임의 컬럼명 정의
columnNameList = ["year", "total", "typhoon", "heavy_rain", "heavy_snow", "heavy_wind", "wind_wave_strong_wind", "typhoon_heavy_rain", "lightning", "cold_wave", "earthquake", "heatwave"]

### 처음 인덱스로 데이터 확인

In [ ]:
year = year_list[0].text
year

'2016'

In [ ]:
total = total_list[0].text
total

'288862'

In [ ]:
typhoon = typhoon_list[0].text
typhoon

'214965'

In [ ]:
heavy_rain = heavy_rain_list[0].text
heavy_rain

'35887'

In [ ]:
heavy_snow = heavy_snow_list[0].text
heavy_snow

'18689'

In [ ]:
heavy_wind = heavy_wind_list[0].text
heavy_wind

'0'

In [ ]:
wind_wave_strong_wind = wind_wave_strong_wind_list[0].text
wind_wave_strong_wind

'8302'

In [ ]:
typhoon_heavy_rain = typhoon_heavy_rain_list[0].text
typhoon_heavy_rain

'0'

In [ ]:
lightning = lightning_list[0].text
lightning

'0'

In [ ]:
cold_wave = cold_wave_list[0].text
cold_wave

'0'

In [ ]:
earthquak = earthquak_list[0].text
earthquak

'11020'

In [ ]:
heatwave = heatwave_list[0].text
heatwave

'0'

In [ ]:
# 각 항목을 한 행으로 리스트에 추가
rowList.append([year, total, typhoon, heavy_rain, heavy_snow, heavy_wind, wind_wave_strong_wind, typhoon_heavy_rain, lightning, cold_wave, earthquak, heatwave])

In [ ]:
rowList

[['2016',
  '288862',
  '214965',
  '35887',
  '18689',
  '0',
  '8302',
  '0',
  '0',
  '0',
  '11020',
  '0']]

### 반복문으로 병합

In [ ]:
rowList = []

for i in range(len(year_list)):
    year = year_list[i].text
    total = total_list[i].text
    typhoon = typhoon_list[i].text
    heavy_rain = heavy_rain_list[i].text
    heavy_snow = heavy_snow_list[i].text
    heavy_wind = heavy_wind_list[i].text
    wind_wave_strong_wind = wind_wave_strong_wind_list[i].text
    typhoon_heavy_rain = typhoon_heavy_rain_list[i].text
    lightning = lightning_list[i].text
    cold_wave = cold_wave_list[i].text
    earthquak = earthquak_list[i].text
    heatwave = heatwave_list[i].text

    rowList.append([year, total, typhoon, heavy_rain, heavy_snow, heavy_wind, wind_wave_strong_wind, typhoon_heavy_rain, lightning, cold_wave, earthquak, heatwave])

In [ ]:
rowList

[['2016',
  '288862',
  '214965',
  '35887',
  '18689',
  '0',
  '8302',
  '0',
  '0',
  '0',
  '11020',
  '0'],
 ['2017',
  '187302',
  '0',
  '101592',
  '83',
  '0',
  '605',
  '0',
  '0',
  '0',
  '85022',
  '0'],
 ['2018',
  '141284',
  '64200',
  '53800',
  '14032',
  '7',
  '2823',
  '6416',
  '0',
  '0',
  '0',
  '7'],
 ['2019',
  '216226',
  '212778',
  '1651',
  '671',
  '7',
  '474',
  '0',
  '26',
  '0',
  '0',
  '618'],
 ['2020',
  '1318177',
  '222541',
  '1095172',
  '0',
  '0',
  '319',
  '0',
  '0',
  '145',
  '0',
  '0'],
 ['2021',
  '66054',
  '21086',
  '40646',
  '3527',
  '445',
  '0',
  '0',
  '52',
  '280',
  '0',
  '0'],
 ['2022',
  '592656',
  '244046',
  '332560',
  '15439',
  '26',
  '585',
  '0',
  '0',
  '0',
  '0',
  '0']]

In [ ]:
# pandas DataFrame으로 변환
disaster_year_result = pd.DataFrame(rowList, columns = columnNameList)
disaster_year_result

,year,total,typhoon,heavy_rain,heavy_snow,heavy_wind,wind_wave_strong_wind,typhoon_heavy_rain,lightning,cold_wave,earthquake,heatwave
0,2016,288862,214965,35887,18689,0,8302,0,0,0,11020,0
1,2017,187302,0,101592,83,0,605,0,0,0,85022,0
2,2018,141284,64200,53800,14032,7,2823,6416,0,0,0,7
3,2019,216226,212778,1651,671,7,474,0,26,0,0,618
4,2020,1318177,222541,1095172,0,0,319,0,0,145,0,0
5,2021,66054,21086,40646,3527,445,0,0,52,280,0,0
6,2022,592656,244046,332560,15439,26,585,0,0,0,0,0


## 연도별 자연재해 복구비 데이터 수집

In [ ]:
# 행정안전부_통계연보_연도별 자연재해 복구비 API URL
cost_year_url = "http://apis.data.go.kr/1741000/AnnualDisasterRecoveryCosts/getAnnualDisasterRecoveryCosts"

In [ ]:
# 쿼리 파라미터에 API_Key 추가
queryParams = '?' + urlencode({
                    quote_plus('ServiceKey') : API_Key,
                })

In [ ]:
# requests 라이브러리를 사용하여 API로부터 XML 데이터를 가져옴
response = requests.get(cost_year_url+queryParams)

# 응답 인코딩을 UTF-8로 설정
response.encoding = "utf-8"

# lxml-xml 파서를 사용하여 BeautifulSoup으로 XML 파싱
cost_year_xmlobj = bs4.BeautifulSoup(response.text, "lxml-xml")
cost_year_xmlobj

<?xml version="1.0" encoding="utf-8"?>
<AnnualDisasterRecoveryCosts>
<head>
<totalCount>16</totalCount>
<numOfRows>100</numOfRows>
<pageNo>1</pageNo>
<type>XML</type>
<RESULT>
<resultCode>INFO-0</resultCode>
<resultMsg>NOMAL SERVICE</resultMsg>
</RESULT>
</head>
<row>
<wrttimeid>2007</wrttimeid>
<seq>1</seq>
<total>545864</total>
<typhoon>408238</typhoon>
<heavy_rain>110421</heavy_rain>
<heavy_snow>3057</heavy_snow>
<strong_wind>3058</strong_wind>
<wind_wave_strong_wind>21090</wind_wave_strong_wind>
<typhoon_heavy_rain>0</typhoon_heavy_rain>
<lightning>0</lightning>
<cold_wave>0</cold_wave>
<earthquake>0</earthquake>
<heatwave>0</heatwave>
</row>
<row>
<wrttimeid>2008</wrttimeid>
<seq>1</seq>
<total>147620</total>
<typhoon>1271</typhoon>
<heavy_rain>144585</heavy_rain>
<heavy_snow>1283</heavy_snow>
<strong_wind>481</strong_wind>
<wind_wave_strong_wind>0</wind_wave_strong_wind>
<typhoon_heavy_rain>0</typhoon_heavy_rain>
<lightning>0</lightning>
<cold_wave>0</cold_wave>
<earthquake>0</ea

### 각 재해 항목별 데이터를 XML에서 추출해 리스트로 저장

In [ ]:
year_list = cost_year_xmlobj.find_all("wrttimeid") # 연도
year_list

[<wrttimeid>2007</wrttimeid>,
 <wrttimeid>2008</wrttimeid>,
 <wrttimeid>2009</wrttimeid>,
 <wrttimeid>2010</wrttimeid>,
 <wrttimeid>2011</wrttimeid>,
 <wrttimeid>2012</wrttimeid>,
 <wrttimeid>2013</wrttimeid>,
 <wrttimeid>2014</wrttimeid>,
 <wrttimeid>2015</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2018</wrttimeid>,
 <wrttimeid>2019</wrttimeid>,
 <wrttimeid>2020</wrttimeid>,
 <wrttimeid>2021</wrttimeid>,
 <wrttimeid>2022</wrttimeid>]

In [ ]:
total_list = cost_year_xmlobj.find_all("total") # 합계
total_list

[<total>545864</total>,
 <total>147620</total>,
 <total>773335</total>,
 <total>715407</total>,
 <total>1654030</total>,
 <total>2053177</total>,
 <total>386560</total>,
 <total>507064</total>,
 <total>38122</total>,
 <total>590607</total>,
 <total>499671</total>,
 <total>443270</total>,
 <total>1348760</total>,
 <total>4161548</total>,
 <total>297322</total>,
 <total>1663681</total>]

In [ ]:
typhoon_list = cost_year_xmlobj.find_all("typhoon") # 태풍
typhoon_list

[<typhoon>408238</typhoon>,
 <typhoon>1271</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>186947</typhoon>,
 <typhoon>461665</typhoon>,
 <typhoon>1932804</typhoon>,
 <typhoon>4297</typhoon>,
 <typhoon>10321</typhoon>,
 <typhoon>28412</typhoon>,
 <typhoon>504907</typhoon>,
 <typhoon>197</typhoon>,
 <typhoon>274288</typhoon>,
 <typhoon>1320328</typhoon>,
 <typhoon>609021</typhoon>,
 <typhoon>104920</typhoon>,
 <typhoon>780308</typhoon>]

In [ ]:
heavy_rain_list = cost_year_xmlobj.find_all("heavy_rain") # 호우
heavy_rain_list

[<heavy_rain>110421</heavy_rain>,
 <heavy_rain>144585</heavy_rain>,
 <heavy_rain>751796</heavy_rain>,
 <heavy_rain>502728</heavy_rain>,
 <heavy_rain>1162842</heavy_rain>,
 <heavy_rain>98960</heavy_rain>,
 <heavy_rain>377674</heavy_rain>,
 <heavy_rain>465569</heavy_rain>,
 <heavy_rain>1621</heavy_rain>,
 <heavy_rain>36809</heavy_rain>,
 <heavy_rain>318852</heavy_rain>,
 <heavy_rain>122179</heavy_rain>,
 <heavy_rain>5373</heavy_rain>,
 <heavy_rain>3552280</heavy_rain>,
 <heavy_rain>188153</heavy_rain>,
 <heavy_rain>860883</heavy_rain>]

In [ ]:
heavy_snow_list = cost_year_xmlobj.find_all("heavy_snow") # 대설
heavy_snow_list

[<heavy_snow>3057</heavy_snow>,
 <heavy_snow>1283</heavy_snow>,
 <heavy_snow>5190</heavy_snow>,
 <heavy_snow>25702</heavy_snow>,
 <heavy_snow>29523</heavy_snow>,
 <heavy_snow>8544</heavy_snow>,
 <heavy_snow>4204</heavy_snow>,
 <heavy_snow>31150</heavy_snow>,
 <heavy_snow>5186</heavy_snow>,
 <heavy_snow>14111</heavy_snow>,
 <heavy_snow>35</heavy_snow>,
 <heavy_snow>6579</heavy_snow>,
 <heavy_snow>398</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>1262</heavy_snow>,
 <heavy_snow>5247</heavy_snow>]

In [ ]:
heavy_wind_list = cost_year_xmlobj.find_all("strong_wind") # 강풍
heavy_wind_list

[<strong_wind>3058</strong_wind>,
 <strong_wind>481</strong_wind>,
 <strong_wind>2442</strong_wind>,
 <strong_wind>1</strong_wind>,
 <strong_wind>0</strong_wind>,
 <strong_wind>12869</strong_wind>,
 <strong_wind>366</strong_wind>,
 <strong_wind>24</strong_wind>,
 <strong_wind>2768</strong_wind>,
 <strong_wind>0</strong_wind>,
 <strong_wind>0</strong_wind>,
 <strong_wind>3</strong_wind>,
 <strong_wind>3</strong_wind>,
 <strong_wind>0</strong_wind>,
 <strong_wind>107</strong_wind>,
 <strong_wind>8</strong_wind>]

In [ ]:
wind_wave_strong_wind_list = cost_year_xmlobj.find_all("wind_wave_strong_wind") # 풍랑·강풍
wind_wave_strong_wind_list

[<wind_wave_strong_wind>21090</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>13907</wind_wave_strong_wind>,
 <wind_wave_strong_wind>29</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>19</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>135</wind_wave_strong_wind>,
 <wind_wave_strong_wind>20265</wind_wave_strong_wind>,
 <wind_wave_strong_wind>564</wind_wave_strong_wind>,
 <wind_wave_strong_wind>2724</wind_wave_strong_wind>,
 <wind_wave_strong_wind>22658</wind_wave_strong_wind>,
 <wind_wave_strong_wind>107</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>17232</wind_wave_strong_wind>]

In [ ]:
typhoon_heavy_rain_list = cost_year_xmlobj.find_all("typhoon_heavy_rain") # 태풍·호우
typhoon_heavy_rain_list

[<typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>37011</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>]

In [ ]:
lightning_list = cost_year_xmlobj.find_all("lightning") # 낙뢰
lightning_list

[<lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>17</lightning>,
 <lightning>0</lightning>]

In [ ]:
cold_wave_list = cost_year_xmlobj.find_all("cold_wave") # 한파
cold_wave_list

[<cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>48</cold_wave>,
 <cold_wave>929</cold_wave>,
 <cold_wave>3</cold_wave>]

In [ ]:
earthquak_list = cost_year_xmlobj.find_all("earthquake") # 지진
earthquak_list

[<earthquake>0</earthquake>,
 <earthquake>0</earthquake>,
 <earthquake>0</earthquake>,
 <earthquake>0</earthquake>,
 <earthquake>0</earthquake>,
 <earthquake>0</earthquake>,
 <earthquake>0</earthquake>,
 <earthquake>0</earthquake>,
 <earthquake>0</earthquake>,
 <earthquake>14515</earthquake>,
 <earthquake>180023</earthquake>,
 <earthquake>0</earthquake>,
 <earthquake>0</earthquake>,
 <earthquake>0</earthquake>,
 <earthquake>0</earthquake>,
 <earthquake>0</earthquake>]

In [ ]:
heatwave_list = cost_year_xmlobj.find_all("heatwave") # 폭염
heatwave_list

[<heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>486</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>92</heatwave>,
 <heatwave>1691</heatwave>,
 <heatwave>0</heatwave>]

In [ ]:
# 행 별로 결과를 저장할 행 리스트 선언
rowList = []

# 데이터프레임의 컬럼명 정의
columnNameList = ["year", "total", "typhoon", "heavy_rain", "heavy_snow", "heavy_wind", "wind_wave_strong_wind", "typhoon_heavy_rain", "lightning", "cold_wave", "earthquake", "heatwave"]

### 각 항목별 값을 추출하여 리스트로 저장

In [ ]:
year = year_list[0].text
total = total_list[0].text
typhoon = typhoon_list[0].text
heavy_rain = heavy_rain_list[0].text
heavy_snow = heavy_snow_list[0].text
heavy_wind = heavy_wind_list[0].text
wind_wave_strong_wind = wind_wave_strong_wind_list[0].text
typhoon_heavy_rain = typhoon_heavy_rain_list[0].text
lightning = lightning_list[0].text
cold_wave = cold_wave_list[0].text
earthquak = earthquak_list[0].text
heatwave = heatwave_list[0].text

rowList.append([year, total, typhoon, heavy_rain, heavy_snow, heavy_wind, wind_wave_strong_wind, typhoon_heavy_rain, lightning, cold_wave, earthquak, heatwave])
rowList

[['2007',
  '545864',
  '408238',
  '110421',
  '3057',
  '3058',
  '21090',
  '0',
  '0',
  '0',
  '0',
  '0']]

### 반복문으로 병합

In [ ]:
rowList = []

for i in range(len(year_list)):
    year = year_list[i].text
    total = total_list[i].text
    typhoon = typhoon_list[i].text
    heavy_rain = heavy_rain_list[i].text
    heavy_snow = heavy_snow_list[i].text
    heavy_wind = heavy_wind_list[i].text
    wind_wave_strong_wind = wind_wave_strong_wind_list[i].text
    typhoon_heavy_rain = typhoon_heavy_rain_list[i].text
    lightning = lightning_list[i].text
    cold_wave = cold_wave_list[i].text
    earthquak = earthquak_list[i].text
    heatwave = heatwave_list[i].text

    rowList.append([year, total, typhoon, heavy_rain, heavy_snow, heavy_wind, wind_wave_strong_wind, typhoon_heavy_rain, lightning, cold_wave, earthquak, heatwave])

In [ ]:
rowList

[['2007',
  '545864',
  '408238',
  '110421',
  '3057',
  '3058',
  '21090',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2008',
  '147620',
  '1271',
  '144585',
  '1283',
  '481',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2009',
  '773335',
  '0',
  '751796',
  '5190',
  '2442',
  '13907',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2010',
  '715407',
  '186947',
  '502728',
  '25702',
  '1',
  '29',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2011',
  '1654030',
  '461665',
  '1162842',
  '29523',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2012',
  '2053177',
  '1932804',
  '98960',
  '8544',
  '12869',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2013',
  '386560',
  '4297',
  '377674',
  '4204',
  '366',
  '19',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2014',
  '507064',
  '10321',
  '465569',
  '31150',
  '24',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2015',
  '38122',
  '28412',
  '1621',
  '5186',
  '2768',
  '135',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2016',
  '590607',
  

In [ ]:
# pandas DataFrame으로 변환
cost_year_result = pd.DataFrame(rowList, columns = columnNameList)
cost_year_result

,year,total,typhoon,heavy_rain,heavy_snow,heavy_wind,wind_wave_strong_wind,typhoon_heavy_rain,lightning,cold_wave,earthquake,heatwave
0,2007,545864,408238,110421,3057,3058,21090,0,0,0,0,0
1,2008,147620,1271,144585,1283,481,0,0,0,0,0,0
2,2009,773335,0,751796,5190,2442,13907,0,0,0,0,0
3,2010,715407,186947,502728,25702,1,29,0,0,0,0,0
4,2011,1654030,461665,1162842,29523,0,0,0,0,0,0,0
5,2012,2053177,1932804,98960,8544,12869,0,0,0,0,0,0
6,2013,386560,4297,377674,4204,366,19,0,0,0,0,0
7,2014,507064,10321,465569,31150,24,0,0,0,0,0,0
8,2015,38122,28412,1621,5186,2768,135,0,0,0,0,0
9,2016,590607,504907,36809,14111,0,20265,0,0,0,14515,0


## 지역별 자연재해 복구비 데이터 수집

In [ ]:
# 행정안전부_통계연보_지역별 자연재해 복구비 API URL
cost_region_url = "http://apis.data.go.kr/1741000/RegionDisasterRecoveryCosts/getRegionDisasterRecoveryCosts"

In [ ]:
# 쿼리 파라미터에 API_Key 추가
queryParams = '?' + urlencode({
                    quote_plus('ServiceKey') : API_Key,
                })

In [ ]:
# requests 라이브러리를 사용하여 API로부터 XML 데이터를 가져옴
response = requests.get(cost_region_url+queryParams)

# 응답 인코딩을 UTF-8로 설정
response.encoding = "utf-8"

# lxml-xml 파서를 사용하여 BeautifulSoup으로 XML 파싱
cost_region_xmlobj = bs4.BeautifulSoup(response.text, "lxml-xml")
cost_region_xmlobj

<?xml version="1.0" encoding="utf-8"?>
<RegionDisasterRecoveryCosts>
<head>
<totalCount>126</totalCount>
<numOfRows>100</numOfRows>
<pageNo>1</pageNo>
<type>XML</type>
<RESULT>
<resultCode>INFO-0</resultCode>
<resultMsg>NOMAL SERVICE</resultMsg>
</RESULT>
</head>
<row>
<wrttimeid>2016</wrttimeid>
<seq>1</seq>
<region>합계</region>
<tot>590607.3</tot>
<typhoon>504906.8</typhoon>
<heavy_rain>36808.9</heavy_rain>
<heavy_snow>14111.5</heavy_snow>
<heavy_wind>0</heavy_wind>
<wind_wave_strong_wind>20265.4</wind_wave_strong_wind>
<typhoon_heavy_rain>0</typhoon_heavy_rain>
<lightning>0</lightning>
<cold_wave>0</cold_wave>
<earthquak>14514.7</earthquak>
<heatwave>0</heatwave>
</row>
<row>
<wrttimeid>2016</wrttimeid>
<seq>2</seq>
<region>서울</region>
<tot>508.3</tot>
<typhoon>0</typhoon>
<heavy_rain>508.3</heavy_rain>
<heavy_snow>0</heavy_snow>
<heavy_wind>0</heavy_wind>
<wind_wave_strong_wind>0</wind_wave_strong_wind>
<typhoon_heavy_rain>0</typhoon_heavy_rain>
<lightning>0</lightning>
<cold_wave>0

### 각 재해 항목별 데이터를 XML에서 추출해 리스트로 저장

In [ ]:
year_list = cost_region_xmlobj.find_all("wrttimeid") # 연도
year_list

[<wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimei

In [ ]:
region_list = cost_region_xmlobj.find_all("region") # 지역
region_list

[<region>합계</region>,
 <region>서울</region>,
 <region>부산</region>,
 <region>대구</region>,
 <region>인천</region>,
 <region>광주</region>,
 <region>대전</region>,
 <region>울산</region>,
 <region>세종</region>,
 <region>경기</region>,
 <region>강원</region>,
 <region>충북</region>,
 <region>충남</region>,
 <region>전북</region>,
 <region>전남</region>,
 <region>경북</region>,
 <region>경남</region>,
 <region>제주</region>,
 <region>합계</region>,
 <region>서울</region>,
 <region>부산</region>,
 <region>대구</region>,
 <region>인천</region>,
 <region>광주</region>,
 <region>대전</region>,
 <region>울산</region>,
 <region>세종</region>,
 <region>경기</region>,
 <region>강원</region>,
 <region>충북</region>,
 <region>충남</region>,
 <region>전북</region>,
 <region>전남</region>,
 <region>경북</region>,
 <region>경남</region>,
 <region>제주</region>,
 <region>합계</region>,
 <region>서울</region>,
 <region>부산</region>,
 <region>대구</region>,
 <region>인천</region>,
 <region>광주</region>,
 <region>대전</region>,
 <region>울산</region>,
 <region>세종</region>,
 <region>경

In [ ]:
total_list = cost_region_xmlobj.find_all("tot") # 합계
total_list

[<tot>590607.3</tot>,
 <tot>508.3</tot>,
 <tot>86221.2</tot>,
 <tot>584.2</tot>,
 <tot>108.5</tot>,
 <tot>303.7</tot>,
 <tot>396.0</tot>,
 <tot>134333.9</tot>,
 <tot>0</tot>,
 <tot>4544.5</tot>,
 <tot>19078.8</tot>,
 <tot>2997.7</tot>,
 <tot>6375.7</tot>,
 <tot>4585.0</tot>,
 <tot>15140.4</tot>,
 <tot>75496.4</tot>,
 <tot>173575.9</tot>,
 <tot>66356.9</tot>,
 <tot>499672.2</tot>,
 <tot>2338.8</tot>,
 <tot>2578.8</tot>,
 <tot>0</tot>,
 <tot>4075.0</tot>,
 <tot>35.0</tot>,
 <tot>2.0</tot>,
 <tot>0</tot>,
 <tot>105.5</tot>,
 <tot>5081.5</tot>,
 <tot>23047.4</tot>,
 <tot>184437.8</tot>,
 <tot>80501.6</tot>,
 <tot>774.0</tot>,
 <tot>49.1</tot>,
 <tot>187621.8</tot>,
 <tot>8169.1</tot>,
 <tot>855.1</tot>,
 <tot>443269.8</tot>,
 <tot>3468.0</tot>,
 <tot>10463.0</tot>,
 <tot>116.0</tot>,
 <tot>733.4</tot>,
 <tot>3128.8</tot>,
 <tot>5193.0</tot>,
 <tot>562.5</tot>,
 <tot>678.7</tot>,
 <tot>43797.9</tot>,
 <tot>13586.4</tot>,
 <tot>24837.7</tot>,
 <tot>4094.7</tot>,
 <tot>12266.2</tot>,
 <tot>72

In [ ]:
typhoon_list = cost_region_xmlobj.find_all("typhoon") # 태풍
typhoon_list

[<typhoon>504906.8</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>86171.7</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>1.5</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>133654.9</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>538.5</typhoon>,
 <typhoon>10848.5</typhoon>,
 <typhoon>40421.7</typhoon>,
 <typhoon>171175.3</typhoon>,
 <typhoon>62094.7</typhoon>,
 <typhoon>197.4</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>197.4</typhoon>,
 <typhoon>274288.5</typhoon>,
 <typhoon>0</typhoon>,
 <typhoon>9826.9</typhoon>,
 <typhoon>17.0</typhoon>,
 <typh

In [ ]:
heavy_rain_list = cost_region_xmlobj.find_all("heavy_rain") # 호우
heavy_rain_list

[<heavy_rain>36808.9</heavy_rain>,
 <heavy_rain>508.3</heavy_rain>,
 <heavy_rain>6.0</heavy_rain>,
 <heavy_rain>584.2</heavy_rain>,
 <heavy_rain>60.5</heavy_rain>,
 <heavy_rain>23.0</heavy_rain>,
 <heavy_rain>396.0</heavy_rain>,
 <heavy_rain>0</heavy_rain>,
 <heavy_rain>0</heavy_rain>,
 <heavy_rain>4495.0</heavy_rain>,
 <heavy_rain>17147.2</heavy_rain>,
 <heavy_rain>2996.7</heavy_rain>,
 <heavy_rain>5158.4</heavy_rain>,
 <heavy_rain>95.3</heavy_rain>,
 <heavy_rain>0</heavy_rain>,
 <heavy_rain>2939.1</heavy_rain>,
 <heavy_rain>2393.2</heavy_rain>,
 <heavy_rain>6.0</heavy_rain>,
 <heavy_rain>318852.1</heavy_rain>,
 <heavy_rain>2338.8</heavy_rain>,
 <heavy_rain>2259.5</heavy_rain>,
 <heavy_rain>0</heavy_rain>,
 <heavy_rain>4075.0</heavy_rain>,
 <heavy_rain>35.0</heavy_rain>,
 <heavy_rain>2.0</heavy_rain>,
 <heavy_rain>0</heavy_rain>,
 <heavy_rain>105.5</heavy_rain>,
 <heavy_rain>5081.5</heavy_rain>,
 <heavy_rain>23032.6</heavy_rain>,
 <heavy_rain>184437.8</heavy_rain>,
 <heavy_rain>80501.

In [ ]:
heavy_snow_list = cost_region_xmlobj.find_all("heavy_snow") # 대설
heavy_snow_list

[<heavy_snow>14111.5</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>35.5</heavy_snow>,
 <heavy_snow>280.7</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>30.0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>1215.8</heavy_snow>,
 <heavy_snow>3951.2</heavy_snow>,
 <heavy_snow>3648.0</heavy_snow>,
 <heavy_snow>694.1</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>4256.1</heavy_snow>,
 <heavy_snow>35.3</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>9.8</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>,
 <heavy_snow>0</heavy_snow>

In [ ]:
heavy_wind_list = cost_region_xmlobj.find_all("heavy_wind") # 강풍
heavy_wind_list

[<heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0</heavy_wind>,
 <heavy_wind>0

In [ ]:
wind_wave_strong_wind_list = cost_region_xmlobj.find_all("wind_wave_strong_wind") # 풍랑·강풍
wind_wave_strong_wind_list

[<wind_wave_strong_wind>20265.4</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>11.0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>19.5</wind_wave_strong_wind>,
 <wind_wave_strong_wind>1931.7</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>1.5</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>641.9</wind_wave_strong_wind>,
 <wind_wave_strong_wind>17659.8</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>0</wind_wave_strong_wind>,
 <wind_wave_strong_wind>564.3</wind_wave_strong_wind>

In [ ]:
typhoon_heavy_rain_list = cost_region_xmlobj.find_all("typhoon_heavy_rain") # 태풍·호우
typhoon_heavy_rain_list

[<typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_heavy_rain>0</typhoon_heavy_rain>,
 <typhoon_

In [ ]:
lightning_list = cost_region_xmlobj.find_all("lightning") # 낙뢰
lightning_list

[<lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 <lightning>0</lightning>,
 

In [ ]:
cold_wave_list = cost_region_xmlobj.find_all("cold_wave") # 한파
cold_wave_list

[<cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 <cold_wave>0</cold_wave>,
 

In [ ]:
earthquak_list = cost_region_xmlobj.find_all("earthquak") # 지진
earthquak_list

[<earthquak>14514.7</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>43.5</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>679.0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>1.0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>2.0</earthquak>,
 <earthquak>13781.7</earthquak>,
 <earthquak>7.5</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>180023.3</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>0</earthquak>,
 <earthquak>180019.8</earthquak>,
 <earthquak>3.5</earthquak>,
 <earthquak>0<

In [ ]:
heatwave_list = cost_region_xmlobj.find_all("heatwave") # 폭염
heatwave_list

[<heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>486.0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwave>,
 <heatwave>0</heatwav

In [ ]:
# 행 별로 결과를 저장할 행 리스트 선언
rowList = []

# 데이터프레임의 컬럼명 정의
columnNameList = ["year", "region", "total", "typhoon", "heavy_rain", "heavy_snow", "heavy_wind", "wind_wave_strong_wind", "typhoon_heavy_rain", "lightning", "cold_wave", "earthquake", "heatwave"]

### 각 항목별 값을 추출하여 리스트로 저장

In [ ]:
year = year_list[0].text
region = region_list[0].text
total = total_list[0].text
typhoon = typhoon_list[0].text
heavy_rain = heavy_rain_list[0].text
heavy_snow = heavy_snow_list[0].text
heavy_wind = heavy_wind_list[0].text
wind_wave_strong_wind = wind_wave_strong_wind_list[0].text
typhoon_heavy_rain = typhoon_heavy_rain_list[0].text
lightning = lightning_list[0].text
cold_wave = cold_wave_list[0].text
earthquak = earthquak_list[0].text
heatwave = heatwave_list[0].text

rowList.append([year, total, typhoon, heavy_rain, heavy_snow, heavy_wind, wind_wave_strong_wind, typhoon_heavy_rain, lightning, cold_wave, earthquak, heatwave])
rowList

[['2016',
  '590607.3',
  '504906.8',
  '36808.9',
  '14111.5',
  '0',
  '20265.4',
  '0',
  '0',
  '0',
  '14514.7',
  '0']]

### 반복문으로 병합

In [ ]:
rowList = []

for i in range(len(year_list)):
    year = year_list[i].text
    region = region_list[i].text
    total = total_list[i].text
    typhoon = typhoon_list[i].text
    heavy_rain = heavy_rain_list[i].text
    heavy_snow = heavy_snow_list[i].text
    heavy_wind = heavy_wind_list[i].text
    wind_wave_strong_wind = wind_wave_strong_wind_list[i].text
    typhoon_heavy_rain = typhoon_heavy_rain_list[i].text
    lightning = lightning_list[i].text
    cold_wave = cold_wave_list[i].text
    earthquak = earthquak_list[i].text
    heatwave = heatwave_list[i].text

    rowList.append([year, region, total, typhoon, heavy_rain, heavy_snow, heavy_wind, wind_wave_strong_wind, typhoon_heavy_rain, lightning, cold_wave, earthquak, heatwave])
rowList

[['2016',
  '합계',
  '590607.3',
  '504906.8',
  '36808.9',
  '14111.5',
  '0',
  '20265.4',
  '0',
  '0',
  '0',
  '14514.7',
  '0'],
 ['2016', '서울', '508.3', '0', '508.3', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['2016',
  '부산',
  '86221.2',
  '86171.7',
  '6.0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '43.5',
  '0'],
 ['2016', '대구', '584.2', '0', '584.2', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['2016',
  '인천',
  '108.5',
  '1.5',
  '60.5',
  '35.5',
  '0',
  '11.0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2016',
  '광주',
  '303.7',
  '0',
  '23.0',
  '280.7',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2016', '대전', '396.0', '0', '396.0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['2016',
  '울산',
  '134333.9',
  '133654.9',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '679.0',
  '0'],
 ['2016', '세종', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['2016',
  '경기',
  '4544.5',
  '0',
  '4495.0',
  '30.0',
  '0',
  '19.5',
  '0',
  '0',
  '0',
  '0',
  '0'

In [ ]:
# pandas DataFrame으로 변환
cost_year_result = pd.DataFrame(rowList, columns = columnNameList)
cost_year_result

,year,region,total,typhoon,heavy_rain,heavy_snow,heavy_wind,wind_wave_strong_wind,typhoon_heavy_rain,lightning,cold_wave,earthquake,heatwave
0,2016,합계,590607.3,504906.8,36808.9,14111.5,0,20265.4,0,0,0,14514.7,0
1,2016,서울,508.3,0,508.3,0,0,0,0,0,0,0,0
2,2016,부산,86221.2,86171.7,6.0,0,0,0,0,0,0,43.5,0
3,2016,대구,584.2,0,584.2,0,0,0,0,0,0,0,0
4,2016,인천,108.5,1.5,60.5,35.5,0,11.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021,광주,171,171,0,0,0,0,0,0,0,0,0
96,2021,대전,4,0,0,0,4,0,0,0,0,0,0
97,2021,울산,3189,3187,0,0,0,0,0,0,2,0,0
98,2021,세종,0,0,0,0,0,0,0,0,0,0,0


## 지역별 자연재해 피해액 데이터 수집

In [ ]:
# 행정안전부_통계연보_지역별 자연재해 피해 API URL
disaster_region_url = "http://apis.data.go.kr/1741000/RegionalNaturalDisasterDamage/getRegionalNaturalDisasterDamage"

In [ ]:
# 쿼리 파라미터에 API_Key 추가
queryParams = '?' + urlencode({
                    quote_plus('ServiceKey') : API_Key,
                })

In [ ]:
# requests 라이브러리를 사용하여 API로부터 XML 데이터를 가져옴
response = requests.get(disaster_region_url+queryParams)

# 응답 인코딩을 UTF-8로 설정
response.encoding = "utf-8"

# lxml-xml 파서를 사용하여 BeautifulSoup으로 XML 파싱
disaster_region_xmlobj = bs4.BeautifulSoup(response.text, "lxml-xml")
disaster_region_xmlobj

<?xml version="1.0" encoding="utf-8"?>
<RegionalNaturalDisasterDamage>
<head>
<totalCount>126</totalCount>
<numOfRows>100</numOfRows>
<pageNo>1</pageNo>
<type>XML</type>
<RESULT>
<resultCode>INFO-0</resultCode>
<resultMsg>NOMAL SERVICE</resultMsg>
</RESULT>
</head>
<row>
<wrttimeid>2016</wrttimeid>
<seq>1</seq>
<region>합계</region>
<prop_tot>288862.5</prop_tot>
<life_tot>7</life_tot>
<typhoon_property_damage>214964.9</typhoon_property_damage>
<typhoon_life_damage>6</typhoon_life_damage>
<heavy_rain_property_damage>35886.8</heavy_rain_property_damage>
<heavy_rain_life_damage>1</heavy_rain_life_damage>
<heavy_snow_property_damage>18688.7</heavy_snow_property_damage>
<heavy_snow_life_damage>0</heavy_snow_life_damage>
<strong_wind_property_damage>0</strong_wind_property_damage>
<strong_wind_life_damage>0</strong_wind_life_damage>
<wind_wave_strong_wind_property_damage>8302.4</wind_wave_strong_wind_property_damage>
<wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>
<typ

### 각 재해 항목별 데이터를 XML에서 추출해 리스트로 저장

In [ ]:
year_list = disaster_region_xmlobj.find_all("wrttimeid") # 연도
year_list

[<wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2016</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimeid>2017</wrttimeid>,
 <wrttimei

In [ ]:
region_list = disaster_region_xmlobj.find_all("region") # 지역
region_list

[<region>합계</region>,
 <region>서울</region>,
 <region>부산</region>,
 <region>대구</region>,
 <region>인천</region>,
 <region>광주</region>,
 <region>대전</region>,
 <region>울산</region>,
 <region>세종</region>,
 <region>경기</region>,
 <region>강원</region>,
 <region>충북</region>,
 <region>충남</region>,
 <region>전북</region>,
 <region>전남</region>,
 <region>경북</region>,
 <region>경남</region>,
 <region>제주</region>,
 <region>합계</region>,
 <region>서울</region>,
 <region>부산</region>,
 <region>대구</region>,
 <region>인천</region>,
 <region>광주</region>,
 <region>대전</region>,
 <region>울산</region>,
 <region>세종</region>,
 <region>경기</region>,
 <region>강원</region>,
 <region>충북</region>,
 <region>충남</region>,
 <region>전북</region>,
 <region>전남</region>,
 <region>경북</region>,
 <region>경남</region>,
 <region>제주</region>,
 <region>합계</region>,
 <region>서울</region>,
 <region>부산</region>,
 <region>대구</region>,
 <region>인천</region>,
 <region>광주</region>,
 <region>대전</region>,
 <region>울산</region>,
 <region>세종</region>,
 <region>경

In [ ]:
prop_tot_list = disaster_region_xmlobj.find_all("prop_tot") # 합계_재산
prop_tot_list

[<prop_tot>288862.5</prop_tot>,
 <prop_tot>220.5</prop_tot>,
 <prop_tot>38408.6</prop_tot>,
 <prop_tot>611.8</prop_tot>,
 <prop_tot>176.2</prop_tot>,
 <prop_tot>67.0</prop_tot>,
 <prop_tot>369.8</prop_tot>,
 <prop_tot>61945.2</prop_tot>,
 <prop_tot>0</prop_tot>,
 <prop_tot>3639.2</prop_tot>,
 <prop_tot>19736.8</prop_tot>,
 <prop_tot>2446.6</prop_tot>,
 <prop_tot>3623.4</prop_tot>,
 <prop_tot>6663.9</prop_tot>,
 <prop_tot>11958.3</prop_tot>,
 <prop_tot>41808.2</prop_tot>,
 <prop_tot>71630.4</prop_tot>,
 <prop_tot>25556.4</prop_tot>,
 <prop_tot>187302.3</prop_tot>,
 <prop_tot>44.6</prop_tot>,
 <prop_tot>1803.4</prop_tot>,
 <prop_tot>0</prop_tot>,
 <prop_tot>2549.6</prop_tot>,
 <prop_tot>2.4</prop_tot>,
 <prop_tot>1.2</prop_tot>,
 <prop_tot>0</prop_tot>,
 <prop_tot>32.4</prop_tot>,
 <prop_tot>2880.5</prop_tot>,
 <prop_tot>7122.0</prop_tot>,
 <prop_tot>57717.5</prop_tot>,
 <prop_tot>24481.9</prop_tot>,
 <prop_tot>71.8</prop_tot>,
 <prop_tot>148.4</prop_tot>,
 <prop_tot>86649.8</prop_tot>,


In [ ]:
life_tot_list = disaster_region_xmlobj.find_all("life_tot") # 합계_인명
life_tot_list

[<life_tot>7</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>2</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>3</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>1</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>1</life_tot>,
 <life_tot>7</life_tot>,
 <life_tot>1</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>1</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>5</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>0</life_tot>,
 <life_tot>53</life_tot>,
 <life_tot>4</life_tot>,
 <life_tot>2</life_tot>,
 <life_tot>3</life_tot>,

In [ ]:
typhoon_prop_list = disaster_region_xmlobj.find_all("typhoon_property_damage") # 태풍_재산
typhoon_prop_list

[<typhoon_property_damage>214964.9</typhoon_property_damage>,
 <typhoon_property_damage>0</typhoon_property_damage>,
 <typhoon_property_damage>38345.0</typhoon_property_damage>,
 <typhoon_property_damage>0</typhoon_property_damage>,
 <typhoon_property_damage>4.3</typhoon_property_damage>,
 <typhoon_property_damage>0</typhoon_property_damage>,
 <typhoon_property_damage>0</typhoon_property_damage>,
 <typhoon_property_damage>61249.8</typhoon_property_damage>,
 <typhoon_property_damage>0</typhoon_property_damage>,
 <typhoon_property_damage>0</typhoon_property_damage>,
 <typhoon_property_damage>0</typhoon_property_damage>,
 <typhoon_property_damage>0</typhoon_property_damage>,
 <typhoon_property_damage>0</typhoon_property_damage>,
 <typhoon_property_damage>0</typhoon_property_damage>,
 <typhoon_property_damage>6747.6</typhoon_property_damage>,
 <typhoon_property_damage>20145.1</typhoon_property_damage>,
 <typhoon_property_damage>68839.8</typhoon_property_damage>,
 <typhoon_property_damage>1

In [ ]:
typhoon_life_list = disaster_region_xmlobj.find_all("typhoon_life_damage") # 태풍_인명
typhoon_life_list

[<typhoon_life_damage>6</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>2</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>3</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>1</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_life_damage>0</typhoon_life_damage>,
 <typhoon_lif

In [ ]:
heavy_rain_prop_list = disaster_region_xmlobj.find_all("heavy_rain_property_damage") # 호우_재산
heavy_rain_prop_list

[<heavy_rain_property_damage>35886.8</heavy_rain_property_damage>,
 <heavy_rain_property_damage>220.5</heavy_rain_property_damage>,
 <heavy_rain_property_damage>1.8</heavy_rain_property_damage>,
 <heavy_rain_property_damage>611.8</heavy_rain_property_damage>,
 <heavy_rain_property_damage>63.8</heavy_rain_property_damage>,
 <heavy_rain_property_damage>1.2</heavy_rain_property_damage>,
 <heavy_rain_property_damage>369.8</heavy_rain_property_damage>,
 <heavy_rain_property_damage>0</heavy_rain_property_damage>,
 <heavy_rain_property_damage>0</heavy_rain_property_damage>,
 <heavy_rain_property_damage>3504.9</heavy_rain_property_damage>,
 <heavy_rain_property_damage>17029.5</heavy_rain_property_damage>,
 <heavy_rain_property_damage>2446.0</heavy_rain_property_damage>,
 <heavy_rain_property_damage>3122.8</heavy_rain_property_damage>,
 <heavy_rain_property_damage>210.7</heavy_rain_property_damage>,
 <heavy_rain_property_damage>0</heavy_rain_property_damage>,
 <heavy_rain_property_damage>5515.8

In [ ]:
heavy_rain_life_list = disaster_region_xmlobj.find_all("heavy_rain_life_damage") # 호우_인명
heavy_rain_life_list

[<heavy_rain_life_damage>1</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>1</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>0</heavy_rain_life_damage>,
 <heavy_rain_life_damage>7</heavy_rain_life_da

In [ ]:
heavy_snow_prop_list = disaster_region_xmlobj.find_all("heavy_snow_property_damage") # 대설_재산
heavy_snow_prop_list

[<heavy_snow_property_damage>18688.7</heavy_snow_property_damage>,
 <heavy_snow_property_damage>0</heavy_snow_property_damage>,
 <heavy_snow_property_damage>0</heavy_snow_property_damage>,
 <heavy_snow_property_damage>0</heavy_snow_property_damage>,
 <heavy_snow_property_damage>91.4</heavy_snow_property_damage>,
 <heavy_snow_property_damage>65.8</heavy_snow_property_damage>,
 <heavy_snow_property_damage>0</heavy_snow_property_damage>,
 <heavy_snow_property_damage>0</heavy_snow_property_damage>,
 <heavy_snow_property_damage>0</heavy_snow_property_damage>,
 <heavy_snow_property_damage>83.7</heavy_snow_property_damage>,
 <heavy_snow_property_damage>0</heavy_snow_property_damage>,
 <heavy_snow_property_damage>0</heavy_snow_property_damage>,
 <heavy_snow_property_damage>500.6</heavy_snow_property_damage>,
 <heavy_snow_property_damage>6453.3</heavy_snow_property_damage>,
 <heavy_snow_property_damage>4991.4</heavy_snow_property_damage>,
 <heavy_snow_property_damage>593.8</heavy_snow_property_

In [ ]:
heavy_snow_life_list = disaster_region_xmlobj.find_all("heavy_snow_life_damage") # 대설_인명
heavy_snow_life_list

[<heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_damage>,
 <heavy_snow_life_damage>0</heavy_snow_life_da

In [ ]:
strong_wind_prop_list = disaster_region_xmlobj.find_all("strong_wind_property_damage") # 강풍_재산
strong_wind_prop_list

[<strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_damage>,
 <strong_wind_property_damage>0</strong_wind_property_d

In [ ]:
strong_wind_life_list = disaster_region_xmlobj.find_all("strong_wind_life_damage") # 강풍_인명
strong_wind_life_list

[<strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_wind_life_damage>0</strong_wind_life_damage>,
 <strong_w

In [ ]:
wind_wave_strong_wind_prop_list = disaster_region_xmlobj.find_all("wind_wave_strong_wind_property_damage") # 풍랑·강풍_재산
wind_wave_strong_wind_prop_list

[<wind_wave_strong_wind_property_damage>8302.4</wind_wave_strong_wind_property_damage>,
 <wind_wave_strong_wind_property_damage>0</wind_wave_strong_wind_property_damage>,
 <wind_wave_strong_wind_property_damage>0</wind_wave_strong_wind_property_damage>,
 <wind_wave_strong_wind_property_damage>0</wind_wave_strong_wind_property_damage>,
 <wind_wave_strong_wind_property_damage>16.7</wind_wave_strong_wind_property_damage>,
 <wind_wave_strong_wind_property_damage>0</wind_wave_strong_wind_property_damage>,
 <wind_wave_strong_wind_property_damage>0</wind_wave_strong_wind_property_damage>,
 <wind_wave_strong_wind_property_damage>0</wind_wave_strong_wind_property_damage>,
 <wind_wave_strong_wind_property_damage>0</wind_wave_strong_wind_property_damage>,
 <wind_wave_strong_wind_property_damage>50.6</wind_wave_strong_wind_property_damage>,
 <wind_wave_strong_wind_property_damage>2707.4</wind_wave_strong_wind_property_damage>,
 <wind_wave_strong_wind_property_damage>0</wind_wave_strong_wind_proper

In [ ]:
wind_wave_strong_wind_life_list = disaster_region_xmlobj.find_all("wind_wave_strong_wind_life_damage") # 풍랑·강풍_인명
wind_wave_strong_wind_life_list

[<wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_life_damage>0</wind_wave_strong_wind_life_damage>,
 <wind_wave_strong_wind_l

In [ ]:
typhoon_heavy_rain_prop_list = disaster_region_xmlobj.find_all("typhoon_heavy_rain_property_damage") # 태풍·호우_재산
typhoon_heavy_rain_prop_list

[<typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,
 <typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,
 <typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,
 <typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,
 <typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,
 <typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,
 <typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,
 <typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,
 <typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,
 <typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,
 <typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,
 <typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,
 <typhoon_heavy_rain_property_damage>0</typhoon_heavy_rain_property_damage>,

In [ ]:
typhoon_heavy_rain_life_list = disaster_region_xmlobj.find_all("typhoon_heavy_rain_life_damage") # 태풍·호우_인명
typhoon_heavy_rain_life_list

[<typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0</typhoon_heavy_rain_life_damage>,
 <typhoon_heavy_rain_life_damage>0

In [ ]:
lightning_prop_list = disaster_region_xmlobj.find_all("lightning_property_damage") # 낙뢰_재산
lightning_prop_list

[<lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage>,
 <lightning_property_damage>0</lightning_property_damage

In [ ]:
lightning_life_list = disaster_region_xmlobj.find_all("lightning_life_damage") # 낙뢰_인명
lightning_life_list

[<lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</lightning_life_damage>,
 <lightning_life_damage>0</ligh

In [ ]:
cold_wave_prop_list = disaster_region_xmlobj.find_all("cold_wave_property_damage") # 한파_재산
cold_wave_prop_list

[<cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage>,
 <cold_wave_property_damage>0</cold_wave_property_damage

In [ ]:
cold_wave_life_list = disaster_region_xmlobj.find_all("cold_wave_life_damage") # 한파_인명
cold_wave_life_list

[<cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold_wave_life_damage>,
 <cold_wave_life_damage>0</cold

In [ ]:
earthquake_prop_list = disaster_region_xmlobj.find_all("earthquake_property_damage") # 지진_재산
earthquake_prop_list

[<earthquake_property_damage>11019.7</earthquake_property_damage>,
 <earthquake_property_damage>0</earthquake_property_damage>,
 <earthquake_property_damage>61.8</earthquake_property_damage>,
 <earthquake_property_damage>0</earthquake_property_damage>,
 <earthquake_property_damage>0</earthquake_property_damage>,
 <earthquake_property_damage>0</earthquake_property_damage>,
 <earthquake_property_damage>0</earthquake_property_damage>,
 <earthquake_property_damage>695.4</earthquake_property_damage>,
 <earthquake_property_damage>0</earthquake_property_damage>,
 <earthquake_property_damage>0</earthquake_property_damage>,
 <earthquake_property_damage>0</earthquake_property_damage>,
 <earthquake_property_damage>0.6</earthquake_property_damage>,
 <earthquake_property_damage>0</earthquake_property_damage>,
 <earthquake_property_damage>0</earthquake_property_damage>,
 <earthquake_property_damage>1.2</earthquake_property_damage>,
 <earthquake_property_damage>10243.9</earthquake_property_damage>,
 

In [ ]:
earthquake_life_list = disaster_region_xmlobj.find_all("earthquake_life_damage") # 지진_인명
earthquake_life_list

[<earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_damage>,
 <earthquake_life_damage>0</earthquake_life_da

In [ ]:
heatwave_prop_list = disaster_region_xmlobj.find_all("heatwave_property_damage") # 폭염_재산
heatwave_prop_list

[<heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</heatwave_property_damage>,
 <heatwave_property_damage>0</h

In [ ]:
heatwave_life_list = disaster_region_xmlobj.find_all("heatwave_life_damage") # 폭염_인명
heatwave_life_list

[<heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_damage>0</heatwave_life_damage>,
 <heatwave_life_dama

In [ ]:
# 행 별로 결과를 저장할 행 리스트 선언
rowList = []

# 데이터프레임의 컬럼명 정의
columnNameList = ["year", "region", "total", "typhoon", "heavy_rain", "heavy_snow", "heavy_wind", "wind_wave_strong_wind", "typhoon_heavy_rain", "lightning", "cold_wave", "earthquake", "heatwave"]

### 각 항목별 값을 추출하여 리스트로 저장

In [ ]:
year = year_list[i].text
region = region_list[i].text
total = str(float(prop_tot_list[i].text) + float(life_tot_list[i].text))

# 각 재해별 총합 (property + life)
typhoon = str(float(typhoon_prop_list[i].text) + float(typhoon_life_list[i].text))
heavy_rain = str(float(heavy_rain_prop_list[i].text) + float(heavy_rain_life_list[i].text))
heavy_snow = str(float(heavy_snow_prop_list[i].text) + float(heavy_snow_life_list[i].text))
heavy_wind = str(float(strong_wind_prop_list[i].text) + float(strong_wind_life_list[i].text))
wind_wave_strong_wind = str(float(wind_wave_strong_wind_prop_list[i].text) + float(wind_wave_strong_wind_life_list[i].text))
typhoon_heavy_rain = str(float(typhoon_heavy_rain_prop_list[i].text) + float(typhoon_heavy_rain_life_list[i].text))
lightning = str(float(lightning_prop_list[i].text) + float(lightning_life_list[i].text))
cold_wave = str(float(cold_wave_prop_list[i].text) + float(cold_wave_life_list[i].text))
earthquake = str(float(earthquake_prop_list[i].text) + float(earthquake_life_list[i].text))
heatwave = str(float(heatwave_prop_list[i].text) + float(heatwave_life_list[i].text))

rowList.append([year, region, total, typhoon, heavy_rain, heavy_snow, heavy_wind, wind_wave_strong_wind, typhoon_heavy_rain, lightning, cold_wave, earthquake, heatwave])
rowList

[['2021',
  '경기',
  '278.0',
  '0.0',
  '83.0',
  '42.0',
  '146.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '7.0']]

### 반복문으로 병합

In [ ]:
rowList = []

for i in range(len(year_list)):
        year = year_list[i].text
        region = region_list[i].text
        total = str(float(prop_tot_list[i].text) + float(life_tot_list[i].text))

        # 각 재해별 총합 (property + life)
        typhoon = str(float(typhoon_prop_list[i].text) + float(typhoon_life_list[i].text))
        heavy_rain = str(float(heavy_rain_prop_list[i].text) + float(heavy_rain_life_list[i].text))
        heavy_snow = str(float(heavy_snow_prop_list[i].text) + float(heavy_snow_life_list[i].text))
        heavy_wind = str(float(strong_wind_prop_list[i].text) + float(strong_wind_life_list[i].text))
        wind_wave_strong_wind = str(float(wind_wave_strong_wind_prop_list[i].text) + float(wind_wave_strong_wind_life_list[i].text))
        typhoon_heavy_rain = str(float(typhoon_heavy_rain_prop_list[i].text) + float(typhoon_heavy_rain_life_list[i].text))
        lightning = str(float(lightning_prop_list[i].text) + float(lightning_life_list[i].text))
        cold_wave = str(float(cold_wave_prop_list[i].text) + float(cold_wave_life_list[i].text))
        earthquake = str(float(earthquake_prop_list[i].text) + float(earthquake_life_list[i].text))
        heatwave = str(float(heatwave_prop_list[i].text) + float(heatwave_life_list[i].text))


        rowList.append([year, region, total, typhoon, heavy_rain, heavy_snow, heavy_wind, wind_wave_strong_wind, typhoon_heavy_rain, lightning, cold_wave, earthquake, heatwave])
rowList

[['2016',
  '합계',
  '288869.5',
  '214970.9',
  '35887.8',
  '18688.7',
  '0.0',
  '8302.4',
  '0.0',
  '0.0',
  '0.0',
  '11019.7',
  '0.0'],
 ['2016',
  '서울',
  '220.5',
  '0.0',
  '220.5',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0'],
 ['2016',
  '부산',
  '38410.6',
  '38347.0',
  '1.8',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '61.8',
  '0.0'],
 ['2016',
  '대구',
  '611.8',
  '0.0',
  '611.8',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0'],
 ['2016',
  '인천',
  '176.2',
  '4.3',
  '63.8',
  '91.4',
  '0.0',
  '16.7',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0'],
 ['2016',
  '광주',
  '67.0',
  '0.0',
  '1.2',
  '65.8',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0'],
 ['2016',
  '대전',
  '369.8',
  '0.0',
  '369.8',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0'],
 ['2016',
  '울산',
  '61948.2',
  '61252.8',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '695.4',
  '

In [ ]:
# pandas DataFrame으로 변환
disaster_region_result = pd.DataFrame(rowList, columns = columnNameList)
disaster_region_result

,year,region,total,typhoon,heavy_rain,heavy_snow,heavy_wind,wind_wave_strong_wind,typhoon_heavy_rain,lightning,cold_wave,earthquake,heatwave
0,2016,합계,288869.5,214970.9,35887.8,18688.7,0.0,8302.4,0.0,0.0,0.0,11019.7,0.0
1,2016,서울,220.5,0.0,220.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016,부산,38410.6,38347.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,61.8,0.0
3,2016,대구,611.8,0.0,611.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,인천,176.2,4.3,63.8,91.4,0.0,16.7,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021,광주,152.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
96,2021,대전,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
97,2021,울산,2458.0,2432.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,2021,세종,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
